### 这只是个占位文件

### 各Demo测试先进入到各项目文件夹可查看

In [1]:
print('我只是个占位文件呀，你为啥要进来看我~')

我只是个占位文件呀，你为啥要进来看我~


In [2]:
print('看够了赶紧出去┭┮﹏┭┮')

看够了赶紧出去┭┮﹏┭┮


In [5]:
print('还看！！！逼我弄一个语法错误吓吓你~')

还看！！！逼我弄一个语法错误吓吓你~


In [6]:
Fuck!!!

SyntaxError: invalid syntax (<ipython-input-6-7460a953df84>, line 1)